# 1. Import Packages

In [ ]:
%load_ext autoreload
%autoreload 2

from util1 import *

In [ ]:
plt.style.use('ggplot')

# 2. Load Dataset

In [ ]:
cached_data_path = join(PATH.cache, 'train_val_test.joblib')
if exists(cached_data_path):
    datas = joblib.load(cached_data_path)

---

In [ ]:
data   = load_data()
target = 'loan_status'
data   = select_data(data, target)
datas  = split_data(data, target, TEST_SIZE)

In [ ]:
sample = datas['train'].sample(100)
plot_missing_value(sample, figsize=(30, 8))
plot_features(sample);

In [ ]:
imputer = get_imputer()
sample = imputer.fit_transform(sample)
plot_missing_value(sample, figsize=(30, 8))
plot_features(sample);

In [ ]:
static_fe_processor = get_static_fe_processor()
sample = static_fe_processor.fit_transform(sample)
plot_missing_value(sample, figsize=(30, 8))
plot_features(sample[[c for c in sample.columns if not c.startswith('emp_title')]], figsize=(30, 20));

In [ ]:
dfe_processor = get_dynamic_fe_processor()
sample = dfe_processor.fit_transform(sample)

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder

pp     = get_preprocessor()
col_tr = pp[1][1]
y_enc  = LabelEncoder()

cached_data = {}
for id in ('train', 'val', 'test'):
    X, y = datas[id].drop(target, axis=1), datas[id][target]
    if id == 'train':
        X_proc = restore_cols(pp.fit_transform(X), col_tr)
        y      = pd.Series(y_enc.fit_transform(y), name=target)
    else:
        X_proc = restore_cols(pp.transform(X), col_tr)
        y      = pd.Series(y_enc.transform(y), name=target)
    cached_data[id] = X_proc.join(y)
cached_data.update({'target': target, 'classes': y_enc.classes_})
joblib.dump(cached_data, join(PATH.cache, 'train_val_test.joblib'))

cached_data = {}
for id in ('train_full', 'test'):
    X, y = datas[id].drop(target, axis=1), datas[id][target]
    if id == 'train_full':
        X_proc = restore_cols(pp.fit_transform(X), col_tr)
        y      = pd.Series(y_enc.fit_transform(y), name=target)
    else:
        X_proc = restore_cols(pp.transform(X), col_tr)
        y      = pd.Series(y_enc.transform(y), name=target)
    cached_data[id] = X_proc.join(y)
cached_data.update({'target': target, 'classes': y_enc.classes_})
joblib.dump(datas, join(PATH.cache, 'train_full_test.joblib'))